In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

from cpscheduler.algorithms.ppo import PPO

In [ ]:
import gymnasium as gym

fn = lambda : gym.make('MountainCar-v0')

obs_shape = (2,)
action_shape = ()

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

agent = nn.Sequential(
    layer_init(nn.Linear(2, 64)),
    nn.Tanh(),
    layer_init(nn.Linear(64, 64)),
    nn.Tanh(),
    layer_init(nn.Linear(64, 3))
)

value = nn.Sequential(
    layer_init(nn.Linear(2, 64)),
    nn.Tanh(),
    layer_init(nn.Linear(64, 64)),
    nn.Tanh(),
    layer_init(nn.Linear(64, 1)),
)

optimizer = optim.Adam([
    {'params': agent.parameters()},
    {'params': value.parameters()}
], lr=1e-4)



ppo = PPO(agent, value, optimizer, fn, obs_shape, action_shape,
          clip_value=False)
ppo.learn(
    num_updates=500000//512,
    steps_per_update=4,
    batch_size=128
)